In [77]:
import urllib.request, json 
import pandas as pd
import numpy as np
import math

In [78]:
data = pd.read_csv("clean_csv/akoya.csv")

managers = ["Ali","Ruslan","Sami","Yahya","Youssef","Santi","Shrey","Dani"]

current_gw = 36 #current gameweek

In [79]:
data.head()

,player_id,player_name,position,team,gameweek,manager_id,squad_position,bench,points,minutes,...,saves,bonus,bps,influence,creativity,threat,ict_index,total_points,in_dreamteam,photo
0,1.0,Cédric,DEF,FUL,5.0,transfer market,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,58822
1,2.0,Leno,GK,FUL,5.0,transfer market,NaN,NaN,2.0,90.0,...,2.0,0.0,13.0,19.0,10.0,0.0,2.9,2.0,False,80201
2,105.0,Duffy,DEF,FUL,5.0,transfer market,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,61933
3,200.0,Cairney,MID,FUL,5.0,transfer market,NaN,NaN,1.0,12.0,...,0.0,0.0,4.0,0.0,1.7,4.0,0.6,1.0,False,76357
4,201.0,Ream,DEF,FUL,5.0,transfer market,NaN,NaN,2.0,90.0,...,0.0,0.0,17.0,16.2,0.7,0.0,1.7,2.0,False,82514


In [80]:
data.columns

Index(['player_id', 'player_name', 'position', 'team', 'gameweek',
       'manager_id', 'squad_position', 'bench', 'points', 'minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'total_points', 'in_dreamteam', 'photo'],
      dtype='object')

# Points

In [81]:
gameweek_points = data[(data["manager_id"]!="transfer market")&(data["bench"]=="fielded")]

grouped = gameweek_points.groupby(["manager_id","gameweek"])["points"].sum()
gameweek_df = grouped.reset_index()
gameweek_df

,manager_id,gameweek,points
0,Ali,5.0,51.0
1,Ali,6.0,43.0
2,Ali,7.0,0.0
3,Ali,8.0,54.0
4,Ali,9.0,53.0
...,...,...,...
251,Youssef,32.0,27.0
252,Youssef,33.0,16.0
253,Youssef,34.0,57.0
254,Youssef,35.0,36.0


In [82]:
def get_ranking(df,bench,goal):
    filtered = df[(df["manager_id"]!="transfer market")&(df["bench"]==bench)&(df["gameweek"]!=7)]
    ranked_df = filtered.groupby(df["manager_id"])
    return ranked_df[goal].sum().sort_values(ascending=False)

In [83]:
get_ranking(data,"fielded","points")

manager_id
Ali        1477.0
Sami       1409.0
Ruslan     1370.0
Yahya      1340.0
Youssef    1304.0
Santi      1221.0
Shrey      1194.0
Dani       1126.0
Name: points, dtype: float64

In [84]:
real_ranking = get_ranking(data,"fielded","points").to_frame()
real_ranking = real_ranking.reset_index().rename(columns={'manager_id': 'manager'})
real_ranking.to_csv(f'findings/points/real_ranking.csv', index=False)

In [85]:
real_ranking.columns

Index(['manager', 'points'], dtype='object')

### League by positions

Make 4 tables, ranking everyone depending on each position (gk, def, mid, fwd)

In [86]:
#GK Table
gk = data[data["position"]=="GK"]
get_ranking(gk,"fielded","points")

manager_id
Shrey      152.0
Ruslan     135.0
Dani       127.0
Sami       126.0
Santi      126.0
Ali        124.0
Youssef    122.0
Yahya      106.0
Name: points, dtype: float64

In [87]:
#DEF Table
deef = data[data["position"]=="DEF"]
get_ranking(deef,"fielded","points")

manager_id
Sami       525.0
Ruslan     434.0
Youssef    404.0
Dani       387.0
Ali        380.0
Yahya      346.0
Santi      305.0
Shrey      303.0
Name: points, dtype: float64

In [88]:
#MID Table
mid = data[data["position"]=="MID"]
get_ranking(mid,"fielded","points")

manager_id
Youssef    611.0
Yahya      580.0
Shrey      570.0
Ali        569.0
Ruslan     527.0
Sami       495.0
Santi      469.0
Dani       412.0
Name: points, dtype: float64

In [89]:
#FWD Table
fwd = data[data["position"]=="FWD"]
get_ranking(fwd,"fielded","points")

manager_id
Ali        404.0
Santi      321.0
Yahya      308.0
Ruslan     274.0
Sami       263.0
Dani       200.0
Shrey      169.0
Youssef    167.0
Name: points, dtype: float64

In [90]:
gks_df = get_ranking(gk,"fielded","points").to_frame()
def_df = get_ranking(deef,"fielded","points").to_frame()
mid_df = get_ranking(mid,"fielded","points").to_frame()
fwd_df = get_ranking(fwd,"fielded","points").to_frame()

gks_df = gks_df.reset_index().rename(columns={'manager_id': 'manager'})
def_df = def_df.reset_index().rename(columns={'manager_id': 'manager'})
mid_df = mid_df.reset_index().rename(columns={'manager_id': 'manager'})
fwd_df = fwd_df.reset_index().rename(columns={'manager_id': 'manager'})

gks_df.to_csv(f'findings/points/gk_ranking.csv', index=False)
def_df.to_csv(f'findings/points/def_ranking.csv', index=False)
mid_df.to_csv(f'findings/points/mid_ranking.csv', index=False)
fwd_df.to_csv(f'findings/points/fwd_ranking.csv', index=False)

### Best gw by position

In [91]:
def get_ranking_gw(df):
    filtered = df[(df["manager_id"]!="transfer market")&(df["bench"]=="fielded")&(df["gameweek"]!=7)]
    ranked_df = filtered.groupby(["manager_id","gameweek"])
    return ranked_df["points"].sum().sort_values(ascending=False)

In [92]:
#GK GW Table
get_ranking_gw(gk)

manager_id  gameweek
Sami        34.0        15.0
Shrey       12.0        15.0
Ali         29.0        14.0
Sami        11.0        14.0
Shrey       11.0        14.0
                        ... 
Dani        30.0         0.0
Youssef     8.0          0.0
            26.0        -1.0
Sami        35.0        -2.0
Ali         24.0        -3.0
Name: points, Length: 248, dtype: float64

In [93]:
#DEF GW Table
get_ranking_gw(deef)

manager_id  gameweek
Sami        36.0        48.0
Ruslan      25.0        47.0
            27.0        41.0
Youssef     25.0        36.0
Sami        29.0        35.0
                        ... 
Shrey       12.0         2.0
Santi       9.0          1.0
Shrey       28.0         1.0
            32.0         0.0
Santi       8.0         -3.0
Name: points, Length: 248, dtype: float64

In [94]:
#MID GW Table
get_ranking_gw(mid)

manager_id  gameweek
Ali         29.0        44.0
Shrey       20.0        42.0
            9.0         41.0
Youssef     23.0        40.0
            14.0        40.0
                        ... 
            33.0         4.0
Dani        32.0         4.0
Ali         32.0         3.0
Dani        25.0         2.0
Ali         28.0         2.0
Name: points, Length: 248, dtype: float64

In [95]:
#FWD GW Table
get_ranking_gw(fwd)

manager_id  gameweek
Santi       36.0        30.0
            34.0        28.0
Ali         9.0         25.0
            5.0         24.0
            8.0         24.0
                        ... 
Youssef     13.0         1.0
Sami        6.0          1.0
Dani        25.0         0.0
Ruslan      19.0         0.0
Santi       8.0          0.0
Name: points, Length: 248, dtype: float64

In [96]:
gks_df = get_ranking_gw(gk).to_frame()
def_df = get_ranking_gw(deef).to_frame()
mid_df = get_ranking_gw(mid).to_frame()
fwd_df = get_ranking_gw(fwd).to_frame()

gks_df = gks_df.reset_index().rename(columns={'manager_id': 'manager'})
def_df = def_df.reset_index().rename(columns={'manager_id': 'manager'})
mid_df = mid_df.reset_index().rename(columns={'manager_id': 'manager'})
fwd_df = fwd_df.reset_index().rename(columns={'manager_id': 'manager'})

gks_df.to_csv(f'findings/points/gw_gk_ranking.csv', index=False)
def_df.to_csv(f'findings/points/gw_def_ranking.csv', index=False)
mid_df.to_csv(f'findings/points/gw_mid_ranking.csv', index=False)
fwd_df.to_csv(f'findings/points/gw_fwd_ranking.csv', index=False)

### Bench FC


In [97]:
get_ranking(data,"bench","points")

manager_id
Ruslan     245.0
Sami       245.0
Santi      232.0
Ali        218.0
Dani       207.0
Shrey      157.0
Youssef    157.0
Yahya      145.0
Name: points, dtype: float64

In [98]:
bench = get_ranking(data,"bench","points").to_frame()

bench = bench.reset_index().rename(columns={'manager_id': 'manager'})

bench_total = bench.groupby("manager").sum().sort_values("points",ascending=False)
bench_total = bench_total.reset_index()

bench.to_csv(f'findings/points/bench.csv', index=False)

### Optimised Bench

In [99]:
def optimise(gw,manager):
    benched = data[(data["gameweek"] == gw)&(data["manager_id"] == manager)&(data["bench"] == "bench")]
    fielded = data[(data["gameweek"] == gw)&(data["manager_id"] == manager)&(data["bench"] == "fielded")]

    points = 0

    positions = [("DEF",3),("MID",2),("FWD",1)]

    b_gk = benched[benched["position"] == "GK"]
    f_gk = fielded[fielded["position"] == "GK"]

    if  f_gk.loc[f_gk.index.tolist()[0],"points"] < b_gk.loc[b_gk.index.tolist()[0],"points"]:
        points += b_gk["points"].item() - f_gk["points"].item()
        benched = benched.drop(index=b_gk.index)

    for indx, b_player in benched.iterrows():
        for pos in positions:
            for indx2, f_player in fielded.iterrows():
                if len(fielded[fielded["position"] == pos[0]]) > pos[1]:
                    if f_player["points"] < b_player["points"]:
                        points += b_player["points"] - f_player["points"]
                        benched = benched.drop(index=indx)
                        fielded = fielded.drop(index=indx2)
                        break  # exit the inner loop and go to the next b_player
            else:
                continue  # only executed if the inner loop didn't break
            break  # exit the outer loop and go to the next b_player

    return points

In [100]:
missed_points = pd.DataFrame(columns=["manager","gameweek","missed_pts"])

for manager in managers:
    for gw in range(5,current_gw):
        new_row = {"manager":manager,"gameweek":gw,"missed_pts":optimise(gw,manager)}
        missed_points = missed_points.append(new_row, ignore_index=True)


C:\Users\santi\AppData\Local\Temp\ipykernel_12856\1275510679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missed_points = missed_points.append(new_row, ignore_index=True)
C:\Users\santi\AppData\Local\Temp\ipykernel_12856\1275510679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missed_points = missed_points.append(new_row, ignore_index=True)
C:\Users\santi\AppData\Local\Temp\ipykernel_12856\1275510679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missed_points = missed_points.append(new_row, ignore_index=True)
C:\Users\santi\AppData\Local\Temp\ipykernel_12856\1275510679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

In [101]:
missed_points

,manager,gameweek,missed_pts
0,Ali,5,1.0
1,Ali,6,5.0
2,Ali,7,0.0
3,Ali,8,2.0
4,Ali,9,0.0
...,...,...,...
243,Dani,31,1.0
244,Dani,32,1.0
245,Dani,33,0.0
246,Dani,34,0.0


In [102]:
missed_points_grouped = missed_points.groupby(missed_points["manager"])
missed_points_grouped["missed_pts"].sum().sort_values(ascending=False)

manager
Sami       155.0
Ruslan     134.0
Santi      132.0
Ali        130.0
Dani       110.0
Shrey      101.0
Youssef     96.0
Yahya       82.0
Name: missed_pts, dtype: float64

In [103]:
bench_best = missed_points_grouped["missed_pts"].sum().sort_values(ascending=False).to_frame()
bench_best = bench_best.reset_index().rename(columns={'missed_pts': 'points'})
bench_best.to_csv(f'findings/points/bench_best.csv', index=False)

### GW Podiums

In [104]:
gw_podiums = pd.DataFrame(columns=["manager","gameweek","points"])

gw_podiums["manager"] = gameweek_df["manager_id"]
gw_podiums["gameweek"] = gameweek_df["gameweek"]
gw_podiums["points"] = gameweek_df["points"]
gw_podiums["rank"] = gw_podiums.groupby("gameweek")["points"].rank(ascending=False).apply(math.floor)
gw_podiums["podium"] = gw_podiums["rank"]<4


gw_podiums

,manager,gameweek,points,rank,podium
0,Ali,5.0,51.0,1,True
1,Ali,6.0,43.0,2,True
2,Ali,7.0,0.0,4,False
3,Ali,8.0,54.0,1,True
4,Ali,9.0,53.0,2,True
...,...,...,...,...,...
251,Youssef,32.0,27.0,4,False
252,Youssef,33.0,16.0,8,False
253,Youssef,34.0,57.0,4,False
254,Youssef,35.0,36.0,4,False


In [105]:
gw_podiums[gw_podiums["manager"] == "Youssef"]
gw_podiums.groupby("manager")["rank"].mean().sort_values()

manager
Ali        3.37500
Sami       3.93750
Yahya      4.00000
Youssef    4.43750
Ruslan     4.50000
Santi      4.81250
Shrey      5.09375
Dani       5.46875
Name: rank, dtype: float64

In [106]:
total_podiums_df = pd.DataFrame(index=['1st', '2nd', '3rd'], columns=gw_podiums['manager'].unique())

grouped = gw_podiums.groupby(['manager', 'rank']).size()

# loop through each manager and count their 1st, 2nd, and 3rd place finishes
for manager in total_podiums_df.columns:
    try:
        total_podiums_df.loc['1st', manager] = grouped[manager, 1]
    except KeyError:
        total_podiums_df.loc['1st', manager] = 0
    try:
        total_podiums_df.loc['2nd', manager] = grouped[manager, 2]
    except KeyError:
        total_podiums_df.loc['2nd', manager] = 0
    try:
        total_podiums_df.loc['3rd', manager] = grouped[manager, 3]
    except KeyError:
        total_podiums_df.loc['3rd', manager] = 0

total_podiums_df.loc['Total'] = total_podiums_df.sum(axis=0)

total_podiums_df

,Ali,Dani,Ruslan,Sami,Santi,Shrey,Yahya,Youssef
1st,8,2,7,7,2,2,3,0
2nd,5,2,2,2,4,4,7,6
3rd,5,3,5,4,5,2,3,6
Total,18,7,14,13,11,8,13,12


In [107]:
total_podiums_df = total_podiums_df.transpose()

In [108]:
total_podiums_df.to_csv(f'findings/points/podiums.csv', index=True,index_label="manager")

### Tottenham Award

Most GW without podium

In [109]:
def longest_streak(df,who):
    current_streak = 1
    max_streak = 1
    indx = 0
    
    df = df[df["manager"]==who]
    
    series = df["podium"]

    for i in range(1, len(series)):
        if series.iloc[i]:
            max_streak = max(max_streak, current_streak)
            current_streak = 1
        else:
            current_streak += 1
            if current_streak > max_streak:
                indx = i

    return max(max_streak, current_streak),df.iloc[indx]["manager"]

In [110]:
longest_nopodium_streak = (0,"")
tottenham = pd.DataFrame(columns=["streak_length"])

for manager in managers:
    streak = longest_streak(gw_podiums, manager)
    if longest_nopodium_streak[0] < streak[0]:
        longest_nopodium_streak = streak
    tottenham.loc[manager,"streak_length"] = streak[0]
    

longest_nopodium_streak

(15, 'Dani')

In [111]:
tottenham = tottenham.sort_values(by="streak_length",ascending=False)

In [112]:
tottenham = tottenham.reset_index().rename(columns={'index': 'manager'})

tottenham.to_csv(f'findings/points/tottenham.csv', index=False)

In [113]:
tottenham

,manager,streak_length
0,Dani,15
1,Shrey,13
2,Sami,7
3,Youssef,7
4,Santi,7
5,Ruslan,5
6,Ali,4
7,Yahya,4


### GW Losers

In [114]:
last_df = pd.DataFrame(index=["Last"], columns=gw_podiums['manager'].unique())

last_grouped = gw_podiums.groupby(['manager', 'rank']).size()

# loop through each manager and count their 1st, 2nd, and 3rd place finishes
for manager in last_df.columns:
    try:
        last_df.loc['Last', manager] = last_grouped[manager, 8]
    except KeyError:
        last_df.loc['1st', manager] = 0


last_df = last_df.transpose().sort_values(by="Last",ascending=False)

In [115]:
last_df = last_df.reset_index().rename(columns={'index': 'manager'})

last_df.to_csv(f'findings/points/last_df.csv', index=False)

# Players

### 1-team Players

In [173]:
def get_loyalty(data):
    grouped = data.groupby(['manager_id',"player_name","team","photo"]).agg({'player_id': 'size', 'points': 'sum'}).sort_values('player_id',ascending=False)
    return grouped

In [174]:
#most owned players per club
no_tm = data[(data["manager_id"]!="transfer market")]
loyalty = get_loyalty(no_tm)
loyalty

player_id  points
manager_id player_name  team photo                    
Yahya      Saka         ARS  223340         32   178.0
Youssef    De Gea       MUN  51940          32   131.0
Ruslan     Ramsdale     ARS  225321         32   120.0
Santi      Pope         NEW  98747          32   129.0
Yahya      Thiago Silva CHE  51090          32    66.0
...                                        ...     ...
Ruslan     Mudryk       CHE  465920          1     0.0
           Mykolenko    EVE  224967          1     1.0
Santi      Olise        CRY  443661          1     4.0
           Neves        WOL  171317          1     2.0
Ali        A.Armstrong  SOU  155511          1     2.0

[437 rows x 2 columns]

In [179]:
loyalty.loc["Youssef"]

,,,player_id,points
player_name,team,photo,,
De Gea,MUN,51940,32,131.0
De Bruyne,MCI,61366,32,154.0
Dias,MCI,171314,32,66.0
Van Dijk,LIV,97032,32,109.0
Martinelli,ARS,444145,32,174.0
Schär,NEW,119471,30,104.0
Almirón,NEW,179018,27,116.0
Johnson,NFO,242898,25,73.0
Havertz,CHE,219847,22,65.0


In [175]:
#most fielded players per club
most_played = get_loyalty(gameweek_points)
most_played

player_id  points
manager_id player_name team photo                    
Yahya      Kane        TOT  78830          32   214.0
Youssef    Martinelli  ARS  444145         32   174.0
Shrey      Fernandes   MUN  141746         31   143.0
Yahya      Ederson     MCI  121160         31   103.0
           Saka        ARS  223340         31   178.0
...                                       ...     ...
Santi      Koulibaly   CHE  86129           1     1.0
Ruslan     Martínez    MUN  221820          1     7.0
Santi      Johnson     NFO  242898          1     2.0
           Jensen      BRE  207283          1     8.0
Ali        A.Armstrong SOU  155511          1     2.0

[401 rows x 2 columns]

### Globe Trotters

Players in the most teams

In [120]:
most_teams_grouped = loyalty.groupby(["player_name","team"]).size().sort_values(axis=0,ascending=False)
most_teams_df = most_teams_grouped.to_frame().sort_values(by=[0, 'player_name'], ascending=[False, True])
most_teams_df[most_teams_df.iloc[:,0]>3]

,,0
player_name,team,
Chilwell,CHE,5
Sancho,MUN,5
Edouard,CRY,4
Eriksen,MUN,4
Gündogan,MCI,4
Martial,MUN,4
Martínez,MUN,4
Walker,MCI,4


### Club Mascot

In [121]:
def get_clubmascot(data):
    mascot_grouped = data.groupby(['manager_id',"team"]).sum().sort_values(by=[0, 'manager_id'], ascending=[False, True])
    return mascot_grouped

In [122]:
#most owned club
club_mascot = get_clubmascot(most_played)
club_mascot["bench"] = get_clubmascot(loyalty)
club_mascot.rename(columns={0: 'fielded'}, inplace=True)
club_mascot["ratio"] = round(club_mascot["fielded"]/club_mascot["bench"],2)
club_mascot

fielded  bench  ratio
manager_id team                       
Ruslan     ARS       190    231   0.82
Ali        MCI        93    116   0.80
Dani       BRE        87    101   0.86
Sami       LIV        67     80   0.84
Youssef    MCI        65     85   0.76
...                  ...    ...    ...
Dani       LEE         1      3   0.33
           WHU         1      2   0.50
Ruslan     EVE         1      1   1.00
           LEI         1      3   0.33
Sami       NFO         1      1   1.00

[131 rows x 3 columns]

In [123]:
club_mascot.loc["Santi"]

,fielded,bench,ratio
team,,,
MUN,45,67,0.67
LIV,43,53,0.81
CHE,36,57,0.63
AVL,35,38,0.92
NEW,29,43,0.67
TOT,23,26,0.88
BHA,22,32,0.69
MCI,22,31,0.71
LEE,18,19,0.95


# Stats

### Most Goals

In [124]:
get_ranking(data,"fielded","goals_scored")

manager_id
Ali        76.0
Yahya      65.0
Ruslan     62.0
Santi      60.0
Sami       52.0
Youssef    48.0
Shrey      42.0
Dani       35.0
Name: goals_scored, dtype: float64

In [125]:
get_ranking(data,"bench","goals_scored")

manager_id
Ruslan     7.0
Ali        6.0
Dani       6.0
Santi      6.0
Shrey      5.0
Youssef    5.0
Sami       2.0
Yahya      2.0
Name: goals_scored, dtype: float64

### Most Assists

In [126]:
get_ranking(data,"fielded","assists")

manager_id
Sami       51.0
Ali        49.0
Shrey      48.0
Yahya      46.0
Youssef    45.0
Ruslan     44.0
Dani       41.0
Santi      26.0
Name: assists, dtype: float64

In [127]:
get_ranking(data,"bench","assists")

manager_id
Sami       6.0
Ruslan     4.0
Youssef    4.0
Ali        3.0
Dani       3.0
Shrey      3.0
Santi      2.0
Yahya      0.0
Name: assists, dtype: float64

### Most Clean Sheets

In [128]:
gk_def = data[(data["position"]=="GK")|(data["position"]=="DEF")]

get_ranking(gk_def,"fielded","clean_sheets")

manager_id
Sami       55.0
Ruslan     51.0
Youssef    50.0
Ali        49.0
Dani       43.0
Yahya      43.0
Santi      39.0
Shrey      36.0
Name: clean_sheets, dtype: float64

In [129]:
get_ranking(gk_def,"bench","clean_sheets")

manager_id
Ruslan     14.0
Sami       13.0
Santi      13.0
Ali        11.0
Dani       10.0
Yahya       9.0
Youssef     9.0
Shrey       8.0
Name: clean_sheets, dtype: float64

### Avg Minutes Played

In [130]:
round(get_ranking(data,"fielded","minutes")/11/31)

manager_id
Ali        84.0
Sami       83.0
Ruslan     82.0
Youssef    81.0
Yahya      80.0
Santi      79.0
Dani       78.0
Shrey      73.0
Name: minutes, dtype: float64

### Maguire Award
Most Goals Conceded

In [131]:
get_ranking(gk_def,"fielded","goals_conceded")

manager_id
Ruslan     188.0
Youssef    185.0
Sami       179.0
Dani       174.0
Ali        172.0
Santi      160.0
Shrey      154.0
Yahya      139.0
Name: goals_conceded, dtype: float64

### 2016 Pessi Award
Most Missed Pens

In [132]:
get_ranking(data,"fielded","penalties_missed")

manager_id
Dani       4.0
Sami       2.0
Santi      2.0
Yahya      2.0
Ruslan     1.0
Shrey      1.0
Youssef    1.0
Ali        0.0
Name: penalties_missed, dtype: float64

### Not De Gea Award
Most Pens Saved

In [133]:
get_ranking(data,"fielded","penalties_saved")

manager_id
Sami       2.0
Ali        1.0
Dani       1.0
Santi      1.0
Shrey      1.0
Ruslan     0.0
Yahya      0.0
Youssef    0.0
Name: penalties_saved, dtype: float64

### Prime Maguire Award
Most Own Goals

In [134]:
get_ranking(data,"fielded","own_goals")

manager_id
Santi      3.0
Dani       2.0
Ali        1.0
Sami       1.0
Shrey      1.0
Youssef    1.0
Ruslan     0.0
Yahya      0.0
Name: own_goals, dtype: float64

### Sergio Ramos Award
Most Red Cards

In [135]:
get_ranking(data,"fielded","red_cards")

manager_id
Ruslan     2.0
Santi      2.0
Sami       1.0
Ali        0.0
Dani       0.0
Shrey      0.0
Yahya      0.0
Youssef    0.0
Name: red_cards, dtype: float64

### Sergio Ramos Lite Award
Most Yellow Cards

In [136]:
get_ranking(data,"fielded","yellow_cards")

manager_id
Dani       50.0
Yahya      48.0
Ali        47.0
Sami       43.0
Santi      35.0
Ruslan     34.0
Shrey      33.0
Youssef    33.0
Name: yellow_cards, dtype: float64

### The Wall Award
Most Saves

In [137]:
get_ranking(data,"fielded","saves")

manager_id
Dani       118.0
Sami       111.0
Shrey      103.0
Ruslan      97.0
Santi       82.0
Youssef     80.0
Ali         76.0
Yahya       41.0
Name: saves, dtype: float64

### BPS Merchant

In [138]:
filtered_bps = data[(data["manager_id"]!="transfer market")&(data["bench"]=="bench")]
df = filtered_bps.groupby(data["manager_id"])
df["bonus"].sum().sort_values(ascending=False)

manager_id
Sami       31.0
Santi      29.0
Shrey      24.0
Dani       20.0
Ruslan     16.0
Yahya      16.0
Ali        15.0
Youssef    13.0
Name: bonus, dtype: float64

### TOTW Merchant

In [139]:
get_ranking(data,"fielded","in_dreamteam")

manager_id
Ali        34
Sami       30
Yahya      28
Ruslan     24
Youssef    18
Shrey      17
Santi      14
Dani       10
Name: in_dreamteam, dtype: object

# Transfers

In [140]:
transfers = pd.read_csv("clean_csv/transfers.csv")

In [141]:
transfers[:10]

,players_id_in,players_id_out,manager_id,gameweek,type,result,transfer_id,player_name_in,photo_in,team_in,player_name_out,photo_out,team_out
0,189,64,Ali,5,f,a,8830907,Gray,172632,EVE,Lerma,152551,BOU
1,346,465,Yahya,5,f,a,8833951,Andreas,156689,FUL,Bowen,178186,WHU
2,134,259,Ruslan,5,f,a,8834214,Kanté,116594,CHE,Tielemans,166989,LEI
3,358,43,Yahya,5,f,a,8838057,Burn,78916,NEW,Cash,199796,AVL
4,342,504,Sami,5,f,a,8857423,Dalot,216051,MUN,Malacia,222690,MUN
5,235,8,Ruslan,5,f,a,8881578,Struijk,222694,LEE,Tierney,192895,ARS
6,344,469,Ali,5,f,a,8918574,Elanga,449434,MUN,Fornals,217593,WHU
7,603,142,Yahya,6,w,di,8969639,Paquetá,224024,WHU,Mount,184341,CHE
8,526,139,Yahya,6,w,a,8969640,Mee,51927,BRE,Chilwell,172850,CHE
9,319,227,Youssef,6,w,a,9210908,Álvarez,461358,MCI,Bamford,106617,LEE


In [142]:
def get_ranking_transfer(df,types):
    if types == "all":
        filtered = df
    else:
        filtered = df[(df["type"]==types)]
    ranked_df = filtered.groupby(df["manager_id"])
    return ranked_df["transfer_id"].size().sort_values(ascending=False)

### Ranking by Total Transfers

In [143]:
accepted = transfers[transfers["result"]=="a"].reset_index()

In [144]:
get_ranking_transfer(accepted,"all")

manager_id
Ali        90
Santi      75
Sami       70
Ruslan     46
Youssef    34
Shrey      26
Yahya       8
Dani        4
Name: transfer_id, dtype: int64

In [145]:
get_ranking_transfer(accepted,"w")

manager_id
Ali        69
Santi      59
Sami       53
Ruslan     33
Youssef    33
Shrey      20
Dani        4
Yahya       4
Name: transfer_id, dtype: int64

In [146]:
get_ranking_transfer(accepted,"f")

manager_id
Ali        21
Sami       17
Santi      16
Ruslan     13
Shrey       6
Yahya       4
Youssef     1
Name: transfer_id, dtype: int64

### Trade Winner & Loser TBD


In [147]:
#get how long a transfer stays at the club, and how long until someone signs him up again
for indx, row in accepted.iterrows():
    player_in = row["players_id_in"]
    player_out = row["players_id_out"]

    for i in range(indx+1, len(accepted)):
        if player_out == accepted.loc[i, "players_id_in"]:
            accepted.loc[indx, "gw_until_in"] = accepted.loc[i, "gameweek"]-accepted.loc[indx, "gameweek"] #gameweeks until someone else signs this player
            accepted.loc[i, "past_manager"] = accepted.loc[indx, "manager_id"] #who owned him before
            accepted.loc[indx, "next_manager"] = accepted.loc[i, "manager_id"] #who will own him next
            break
    
    for i in range(indx+1, len(accepted)):
        if player_in == accepted.loc[i, "players_id_out"]:
            accepted.loc[indx, "gw_until_out"] = accepted.loc[i, "gameweek"]-accepted.loc[indx, "gameweek"] #gameweeks until this player leaves the team
            break
        

In [148]:
def get_points(player_id,gw):
    return data[(data["player_id"]==player_id)&(data["gameweek"]==gw)]["points"].iloc[0]

In [149]:
for indx, row in accepted.iterrows():
    gw = row["gameweek"]
    gws_out = row["gw_until_out"]
    player_in = row["players_id_in"]
    player_out = row["players_id_out"]

    points_in = 0
    points_out = 0

    
    if gws_out >= 3:
        for i in range(1,4):
            x = i - 1
            points_in_gw = get_points(player_in,gw + x)
            points_out_gw = get_points(player_out,gw + x)

            points_in += points_in_gw
            points_out += points_out_gw
    elif math.isnan(gws_out) == False:
        for i in range(1,int(gws_out+1)):
            x = i - 1
            points_in_gw = get_points(player_in,gw + x)
            points_out_gw = get_points(player_out,gw + x)

            points_in += points_in_gw
            points_out += points_out_gw

    accepted.loc[indx,"in_points"] = points_in
    accepted.loc[indx,"out_points"] = points_out
    accepted.loc[indx,"net_points"] = points_in - points_out

In [150]:
new_order = ['index', 'players_id_in', 'player_name_in', 'team_in', 'in_points', 'past_manager' , 'gw_until_out', 
            'manager_id', 'gameweek', 'type', 'result', 'transfer_id', 'players_id_out',
            'player_name_out', 'team_out', 'out_points', "next_manager", 'gw_until_in', 'net_points']
accepted = accepted.reindex(columns=new_order)
accepted

,index,players_id_in,player_name_in,team_in,in_points,past_manager,gw_until_out,manager_id,gameweek,type,result,transfer_id,players_id_out,player_name_out,team_out,out_points,next_manager,gw_until_in,net_points
0,0,189,Gray,EVE,2.0,NaN,1.0,Ali,5,f,a,8830907,64,Lerma,BOU,3.0,NaN,NaN,-1.0
1,1,346,Andreas,FUL,0.0,NaN,NaN,Yahya,5,f,a,8833951,465,Bowen,WHU,0.0,Santi,1.0,0.0
2,2,134,Kanté,CHE,0.0,NaN,1.0,Ruslan,5,f,a,8834214,259,Tielemans,LEI,2.0,Ali,4.0,-2.0
3,3,358,Burn,NEW,0.0,NaN,NaN,Yahya,5,f,a,8838057,43,Cash,AVL,0.0,Santi,22.0,0.0
4,4,342,Dalot,MUN,10.0,NaN,23.0,Sami,5,f,a,8857423,504,Malacia,MUN,7.0,Ali,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,534,92,Pinnock,BRE,0.0,Santi,NaN,Youssef,37,w,a,52682178,489,Toti,WOL,0.0,NaN,NaN,0.0
349,535,88,Jensen,BRE,0.0,Sami,NaN,Youssef,37,w,a,52682179,703,Danilo,NFO,0.0,NaN,NaN,0.0
350,536,304,Sterling,CHE,0.0,Sami,NaN,Ruslan,37,w,a,52819096,373,Willock,NEW,0.0,NaN,NaN,0.0
351,537,534,McNeil,EVE,0.0,NaN,NaN,Santi,37,f,a,53216206,365,Murphy,NEW,0.0,NaN,NaN,0.0


In [151]:
def get_transfer(data):
    filtering = data[["gameweek",'manager_id',"net_points","player_name_in","team_in","player_name_out","team_out","in_points","out_points","gw_until_out"]]
    return filtering.sort_values(by=["net_points", 'manager_id'], ascending=[False, True])

In [152]:
get_transfer(accepted)[:15]

,gameweek,manager_id,net_points,player_name_in,team_in,player_name_out,team_out,in_points,out_points,gw_until_out
134,19,Youssef,32.0,March,BHA,Luis Díaz,LIV,32.0,0.0,17.0
241,27,Ruslan,17.0,Chilwell,CHE,Walker,MCI,18.0,1.0,5.0
204,23,Santi,17.0,Emerson Royal,TOT,Walker,MCI,23.0,6.0,6.0
78,13,Ali,16.0,Ward,LEI,Neto,BOU,17.0,1.0,5.0
151,20,Ruslan,16.0,Ward-Prowse,SOU,Smith Rowe,ARS,17.0,1.0,2.0
251,28,Santi,16.0,Harrison,LEE,Wissa,BRE,21.0,5.0,7.0
81,13,Sami,15.0,Justin,LEI,Jonny,WOL,15.0,0.0,3.0
239,27,Sami,15.0,Jensen,BRE,Mudryk,CHE,22.0,7.0,6.0
285,31,Ruslan,14.0,Bowen,WHU,Mount,CHE,15.0,1.0,5.0
126,18,Sami,14.0,Leno,FUL,Pickford,EVE,19.0,5.0,15.0
